# Week 1: Introduction and Overview
This notebook accompanies the week 1 lecture

In [4]:
# setup
import sys
import subprocess
import pkg_resources
import pandas as pd
from collections import Counter
import re


required = {'spacy', 'scikit-learn', 'spacy-transformers'}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

import spacy
import transformers
from sklearn.feature_extraction.text import CountVectorizer

## Natural Language Tool Kit vs SpaCy
For this course, we will mainly be using [SpaCy]().  There are a [number of other]() NLP libraries and probably one of the best known is the [Natural Language Tool Kit (NLTK)]().  SpaCy and NLTK both are very powerful, but here I'll show a couple of reasons why I prefer SpaCy.

Note: You will not be able to run these on your own without installing NLTK.  Since it's not used in the rest of the course, I'm not configuring it here.

In [7]:
# spacy
from spacy.lang.en import English
en = English()
text = 'We are doing NLP.'
doc = en(text)
print(type(doc))
print([(x, type(x)) for x in doc])

<class 'spacy.tokens.doc.Doc'>
[(We, <class 'spacy.tokens.token.Token'>), (are, <class 'spacy.tokens.token.Token'>), (doing, <class 'spacy.tokens.token.Token'>), (NLP, <class 'spacy.tokens.token.Token'>), (., <class 'spacy.tokens.token.Token'>)]


In [4]:
# nltk
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
doc = word_tokenize(text)
print(type(doc))
print([(x, type(x)) for x in doc])

<class 'list'>
[('We', <class 'str'>), ('are', <class 'str'>), ('doing', <class 'str'>), ('NLP', <class 'str'>), ('.', <class 'str'>)]


[nltk_data] Downloading package punkt to /Users/batorsky/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
# time comparison
%timeit en(text)
%timeit nltk.tokenize.casual_tokenize(text)

6.63 µs ± 131 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
9.58 µs ± 312 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [6]:
# preprocessing: lower case and removing non-alpha
text = 'We are doing NLP.'
# spacy
%timeit  [x.lower_ for x in en(text) if x.is_alpha]
# nltk (one possible way)
%timeit nltk.RegexpTokenizer(r'\w+').tokenize(text.lower())

9.51 µs ± 180 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.51 µs ± 103 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


You can see both libraries are very powerful.  But SpaCy's syntax is a bit simpler and it's generally a bit faster.  Have you tried NLTK or other libraries? Let's discuss.

### SpaCy's language models
We'll cover this more extensively in the slides.  But here's some illustration of what's going on under the hood with SpaCy

In [12]:
# taking a look at the different languages supported by spacy
#from spacy.lang import 

In [22]:
# what's in a language model
from spacy.lang.en import English
en = English()
print(en.tokenizer)
print(en.pipe_names)

[]


In [21]:
# read in English model with tagging/entity pipeline components
# you will need to run the line below beforehand
#!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
print(nlp.pipe_names)

['tagger', 'parser', 'ner']


In [37]:
# turning text into a spacy document
doc = en('We are doing NLP')
print(doc)
print(type(doc))
doc_attrs = set(dir(doc))
print(doc_attrs)
# spans: subsets of doc
print(doc[:2])
print(type(doc[:2]))
span_attrs = set(dir(doc[:2]))
print(span_attrs - doc_attrs)
# tokens: individual units of doc
print(doc[0])
print(type(doc[0]))
print(dir(doc[0]))

We are doing NLP
<class 'spacy.tokens.doc.Doc'>
{'has_vector', 'merge', 'vocab', '__getattribute__', '__str__', '__reduce__', 'char_span', '_realloc', '__setattr__', 'sentiment', 'extend_tensor', 'to_array', '__new__', 'noun_chunks', '__lt__', '__format__', '__reduce_ex__', 'to_utf8_array', '__setstate__', 'sents', 'is_parsed', 'to_disk', 'from_bytes', 'print_tree', '__pyx_vtable__', 'is_sentenced', 'ents', '_vector_norm', 'is_nered', 'is_tagged', '__init_subclass__', 'from_array', 'count_by', '__iter__', 'cats', 'to_json', 'lang_', '__hash__', 'retokenize', 'vector_norm', 'get_lca_matrix', '__init__', 'to_bytes', 'get_extension', 'mem', '__unicode__', '__gt__', 'tensor', 'user_token_hooks', 'doc', 'vector', '__getitem__', 'user_span_hooks', 'has_extension', '__le__', '__len__', 'set_extension', '__dir__', '__subclasshook__', 'similarity', 'text_with_ws', 'user_hooks', '__doc__', '__ne__', '_vector', 'remove_extension', '_bulk_merge', '_py_tokens', 'lang', 'noun_chunks_iterator', 'text

In [44]:
# internally, spacy represents tokens as hash values
print(doc[0].lower)
en.vocab[doc[0].lower].text
# but you probably won't need that often
print(doc[0].lower_)

16064069575701507746
we


## Tokenization
This section shows some of the considerations to make when tokenizing your data.

Token = "Useful semantic unit"

But what does that mean? This section will detail some considerations here.

In [10]:
# importing different languages in spacy
# blank English model
from spacy.lang.en import English
en = English()
# blank Chinese model
# to run, will need to install jieba tokenizer (optional)
#!pip install jieba
from spacy.lang.zh import Chinese

zh = spacy.lang.zh.Chinese()
zh_text = '我们正在做NLP。'
print('Tokenize in Chinese:', [x.text for x in zh(zh_text)])
print('Tokenize in English:', [x.text for x in en(zh_text)])

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/8b/z4x642j15_1dbkmc_l_p8r540000gn/T/jieba.cache
Loading model cost 0.808 seconds.
Prefix dict has been built successfully.


Tokenize in Chinese: ['我们', '正在', '做', 'NLP', '。']
Tokenize in English: ['我们正在做NLP', '。']


In [49]:
# interesting tidbit: the base Chinese model uses the jieba tokenizer
zh.tokenizer
# any other languages that might require a custom tokenization strategy?

In [ ]:
# lowercasing
text = 'We are doing NLP.'
print('Base python: ', text.lower())
print('SpaCy:', [x.lower_ for x in en(text)])

In [ ]:
# handling non-alpha
text = 'We are doing NLP.'
# base python
strip_punct = '[^A-Za-z0-9 ]'
print(re.sub(strip_punct, '', text))
# spacy
print([x.text for x in en(text) if x.is_alpha])

In [ ]:
# but what about contractions?
text = "We're doing NLP."
# base python
strip_punct = '[^A-Za-z0-9 ]'
print('Just removing punctuation:', re.sub(strip_punct, '', text))
# spacy
print('Removing non-alpha', [x.text for x in en(text) if x.is_alpha])

You can see here that the is_alpha flag is False for any tokens that have non-alpha characters.  We'll look into a better way for dealing with contractions later.

### Exercise: Create a tokenizer
In this exercise, you will make a function that uses spaCy's base English model to tokenize a dataset according to specific parameters.  The functions will take a list of documents and output a list of tokens.  In this case we're interested in outputting strings, rather than spaCy tokens.

In [ ]:
# data
text_data = ["I'm taking a course at Harvard.",
            "I'm learning about Natural Language Processing.",
            "We are studying tokenization, vectorization and modelling.",
            "Check out the course on Github: https://github.com/bpben/nlp_lessons"]

In [ ]:
# initialize model
en = English()

def tokenize_base(docs, model=en):
    # tokenizer that just parses using spaCy's base model
    tokenized_docs = []
    for d in docs:
        parsed = model(d)
        tokenized_docs.append([t.text for t in parsed])
    return(tokenized_docs)

def tokenize_lower_alpha(docs, model=en):
    # tokenizer that lowercases and removes any non-alpha character
    tokenized_docs = []
    for d in docs:
        parsed = model(d)
        tokenized_docs.append([t.lower_ for t in parsed if t.is_alpha])
    return(tokenized_docs)

def tokenize_lower_alpha_url(docs, model=en):
    # tokenizer that lowercases, removes any non-alpha character and removes urls
    tokenized_docs = []
    for d in docs:
        parsed = model(d)
        tokenized_docs.append([t.lower_ for t in parsed if (t.is_alpha)&(not t.like_url)])
    return(tokenized_docs)

### Lemmatization and Stemming
Though word tense can sometimes carry with it a lot of useful information, a lot of time it might be useful to reduce words to their common root.  For example, the word "be" has various forms like "are", "is", "been".  We might not want our vocabulary to contain all these forms and rather count them all as instances of "be".

In [ ]:
# read in English model with tagging/entity pipeline components
# you will need to run the line below beforehand
#!python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')
text = 'I am taking an NLP course.'
print(text)
print([x.lemma_ for x in nlp(text)])

In [ ]:
# guess how lemmatization might change this
text = 'We are recording this class with Zoom.'
guess = ''
print(text)
print([x.lemma_ for x in nlp(text)])

### Stop words
Dealing with stop words involves making some pretty impactful decisions with your data.  Refer to the slides for details.  Here, we just remove stop words based on [spaCy's default set](https://github.com/explosion/spaCy/blob/master/spacy/lang/en/stop_words.py).

In [ ]:
en = English()
text = 'In June 2020, I took a course at Harvard Extension School in Cambridge.'
print(text)
print([x.text for x in en(text) if not x.is_stop])

### Non-standard tokens (e.g. named-entities)
In text, some some n-grams should not be treated as a concatenation of unigrams.  For example, New York City is fundamentally different from the individual words "new", "york" and "city".

Here we attempt to deal with some of these non-standard tokens

In [ ]:
# urls
# base python
# regex from textacy: https://github.com/chartbeat-labs/textacy
SHORT_URL_REGEX = re.compile(
    r"(?:^|(?<![\w/.]))"
    # optional scheme
    r"(?:(?:https?://)?)"
    # domain
    r"(?:\w-?)*?\w+(?:\.[a-z]{2,12}){1,3}"
    r"/+",
    flags=re.IGNORECASE)
text = 'Check out these courses: https://www.summer.harvard.edu/'
print(text)
print(SHORT_URL_REGEX.sub('', text))
# spacy
print([x for x in en(text) if not x.like_url])
# spacy - replace with a standard token
print(['-URL-' if x.like_url else x for x in en(text)])

In [ ]:
# named-entities
# read in English model with tagging/entity pipeline components
nlp = spacy.load('en_core_web_sm')
text = 'I am taking an NLP course at Harvard starting July 19th, 2020'
parsed = nlp(text)
# look at the individual tokens
tokens = [t for t in parsed]
print(tokens)
# look at the identified named-entities and their types
for e in parsed.ents:
    print(e, type(e), e.label_, spacy.explain(e.label_))

### Exercise: A comprehensive tokenization pipeline

In [ ]:
# data
text_data = ["I'm taking a course at Harvard.",
            "I'm learning about Natural Language Processing.",
            "We are studying tokenization, vectorization and modelling.",
            "Check out the course on Github: https://github.com/bpben/nlp_lessons"]

In [ ]:
# initialize model
nlp = spacy.load('en_core_web_sm')

def tokenize_full(docs, model=nlp, 
                  entities=False, 
                  stop_words=False, 
                  lowercase=True, 
                  alpha_only=True, 
                  lemma=True):
    """Full tokenizer with flags for processing steps
    entities: If False, replaces with entity type
    stop_words: If False, removes stop words
    lowercase: If True, lowercases all tokens
    alpha_only: If True, removes all non-alpha characters
    lemma: If True, lemmatizes words
    """
    tokenized_docs = []
    for d in docs:
        parsed = model(d)
        # token collector
        tokens = []
        # index pointer
        i = 0
        # entity collector
        ent = ''
        for t in parsed:
            # only need this if we're replacing entities
            if not entities:
                # replace URLs
                if t.like_url:
                    tokens.append('URL')
                    continue
                # if there's entities collected and current token is non-entity
                if (t.ent_iob_=='O')&(ent!=''):
                    tokens.append(ent)
                    ent = ''
                    continue
                elif t.ent_iob_!='O':
                    ent = t.ent_type_
                    continue
            # only include stop words if stop words==True
            if (t.is_stop)&(not stop_words):
                continue
            # only include non-alpha is alpha_only==False
            if (not t.is_alpha)&(alpha_only):
                continue
            if lemma:
                t = t.lemma_
            else:
                t = t.text
            if lowercase:
                t.lower()
            tokens.append(t)
        tokenized_docs.append(tokens)
    return(tokenized_docs)

In [ ]:
tokenize_full(text_data, stop_words=True, alpha_only=False, entities=True)

## Word counts
A very basic way to use a sanitized list of tokens is to do a word count.  This unlocks a lot of insights right off and is an important step in exploratory data analysis in text.

In [ ]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from spacy.lang.en import English
en = English()

def simple_tokenizer(doc, model=en):
    # a simple tokenizer for individual documents (different from above)
    tokenized_docs = []
    parsed = model(doc)
    return([t.lower_ for t in parsed if (t.is_alpha)&(not t.like_url)])

In [247]:
# data
text_data = ["I'm taking a course at Harvard.",
            "I'm learning about Natural Language Processing.",
            "We are studying tokenization, vectorization and modelling.",
            "Check out the course on Github: https://github.com/bpben/nlp_lessons"]
tokenized = [simple_tokenizer(doc) for doc in text_data]

In [ ]:
# base python: create an make use of a Counter object
counts = [Counter(d) for d in tokenized]
print('List of counts:', counts)
# sum together all counts
all_counts = Counter()
for d in tokenized:
    all_counts += Counter(d)
print(counts)
print('\nCombined count:', all_counts)

In [245]:
# scikit-learn's countvectorizer
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [248]:
# use our custom tokenizer
cv = CountVectorizer(tokenizer=simple_tokenizer)
# outputs sparse array, want to use a normal numpy array
v = cv.fit_transform(text_data).toarray()
# get_feature_names gets the vocabulary of the vectorizer in order
dict(zip(cv.get_feature_names(), v.sum(axis=0)))
# result is the same as above

{'a': 1,
 'about': 1,
 'and': 1,
 'are': 1,
 'at': 1,
 'check': 1,
 'course': 2,
 'github': 1,
 'harvard': 1,
 'i': 2,
 'language': 1,
 'learning': 1,
 'modelling': 1,
 'natural': 1,
 'on': 1,
 'out': 1,
 'processing': 1,
 'studying': 1,
 'taking': 1,
 'the': 1,
 'tokenization': 1,
 'vectorization': 1,
 'we': 1}

In [250]:
# other neat uses of CountVectorizer
# turning it into a dataframe for easier manipulation
cv = CountVectorizer()
v = cv.fit_transform(text_data).toarray()
pd.DataFrame(v, columns=cv.get_feature_names())

,about,and,are,at,bpben,check,com,course,github,harvard,...,nlp_lessons,on,out,processing,studying,taking,the,tokenization,vectorization,we
0,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,1
3,0,0,0,0,1,1,1,1,2,0,...,1,1,1,0,0,0,1,0,0,0


In [251]:
# character-level
cv = CountVectorizer(analyzer='char')
v = cv.fit_transform(text_data).toarray()
pd.DataFrame(v, columns=cv.get_feature_names())

,,',",",.,/,:,_,a,b,c,...,o,p,r,s,t,u,v,w,y,z
0,5,1,0,1,0,0,0,5,0,1,...,1,0,3,1,2,1,1,0,0,0
1,5,1,0,1,0,0,0,6,1,1,...,2,1,3,2,2,3,0,0,0,0
2,6,0,1,1,0,0,0,4,0,1,...,5,0,2,1,5,1,1,1,1,2
3,6,0,0,1,4,2,1,0,4,4,...,5,3,1,5,6,4,0,0,0,0


In [252]:
# n-grams
cv = CountVectorizer(ngram_range=(1,2))
v = cv.fit_transform(text_data).toarray()
pd.DataFrame(v, columns=cv.get_feature_names())

,about,about natural,and,and modelling,are,are studying,at,at harvard,bpben,bpben nlp_lessons,...,taking,taking course,the,the course,tokenization,tokenization vectorization,vectorization,vectorization and,we,we are
0,0,0,0,0,0,0,1,1,0,0,...,1,1,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,1,1,1,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
3,0,0,0,0,0,0,0,0,1,1,...,0,0,1,1,0,0,0,0,0,0


In [253]:
# pre-specified vocabulary
cv = CountVectorizer(vocabulary=['natural', 'language', 'processing', 'harvard'])
v = cv.fit_transform(text_data).toarray()
pd.DataFrame(v, columns=cv.get_feature_names())

,natural,language,processing,harvard
0,0,0,0,1
1,1,1,1,0
2,0,0,0,0
3,0,0,0,0


### Exercise: Sentiment analysis with word counts
Imagine you are a hot dog restaurant owner and you want to analyze a corpus of reviews from diners to see whether people generally think your hot dogs are "good" or "bad".  Specifically, you're going to count up the number of times the word "good" and word "bad" appears.  Depending on how you process the text, you will arrive at different conclusions.  Try a couple ways to see what I mean.

You might also want to think about whether all the reviews are relevant.  Those sorts of choices may also affect your results.  Is there an automatic way you can remove non-relevant reviews?

In [172]:
reviews = ['These hot dogs are really good.',
          'These hot dogs are really bad.',
          'Good hot dogs!',
          'The hot dogs pair well with a Good Humor bar.',
          "I didn't eat anything, I felt bad.",
          "I had a good time!"]

In [210]:
# initialize model
en = English()
def simple_tokenizer(doc, model=en):
    # a simple tokenizer for individual documents (different from above)
    tokenized_docs = []
    parsed = model(doc)
    return([t.lower_ for t in parsed if (t.is_alpha)&(not t.like_url)])

In [174]:
cv = CountVectorizer(tokenizer=simple_tokenizer)
v = cv.fit_transform(reviews).toarray()
count_df = pd.DataFrame(v, columns=cv.get_feature_names())
print('Good count:', count_df['good'].sum())
print('Bad count:', count_df['bad'].sum())

Good count: 4
Bad count: 2


In [171]:
count_df

,a,anything,are,bad,bar,did,dogs,eat,felt,good,...,hot,humor,i,pair,really,the,these,time,well,with
0,0,0,1,0,0,0,1,0,0,1,...,1,0,0,0,1,0,1,0,0,0
1,0,0,1,1,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,1,0,0,1,...,1,1,0,1,0,1,0,0,1,1
4,0,1,0,1,0,1,0,1,1,0,...,0,0,2,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0


In [238]:
cv = CountVectorizer(lowercase=False,
                     # to make it comparable, need to ensure single-character tokens don't get removed,
                     token_pattern=r"(?u)\b\w+\b")
v = cv.fit_transform(reviews).toarray()
count_df = pd.DataFrame(v, columns=cv.get_feature_names())
print('Good count:', count_df['good'].sum())
print('Bad count:', count_df['bad'].sum())

Good count: 2
Bad count: 2


In [239]:
count_df

,Good,Humor,I,The,These,a,anything,are,bad,bar,...,felt,good,had,hot,pair,really,t,time,well,with
0,0,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,1,0,0,0,0
1,0,0,0,0,1,0,0,1,1,0,...,0,0,0,1,0,1,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,1,...,0,0,0,1,1,0,0,0,1,1
4,0,0,2,0,0,0,1,0,1,0,...,1,0,0,0,0,0,1,0,0,0
5,0,0,1,0,0,1,0,0,0,0,...,0,1,1,0,0,0,0,1,0,0


### Simple statistical test: Log-likelihood ratio
Above we just compared the count of the word good to the count of the word bad.  But we can actually test the significance of this difference using a test of log-likelihood ratio.  Refer to the slides for a bit more information, but here's a calculation based on the above example

In [240]:
from numpy import log, mean
def log_likelihood(analysis, reference, word):
    # count of word in source
    a = analysis[word].sum()
    # count of word in reference
    b = reference[word].sum()
    # count of all words in source
    c = analysis.sum().sum()
    # count of all words in reference
    d = reference.sum().sum()
    print('counts analysis:', a)
    print('counts reference:', b)
    e1 = c*(a+b)/(c+d)
    e2 = d*(a+b)/(c+d)
    g = 2*((a*log(a/e1)) + (b*log(b/e2)))
    print('G2: ', g)

In [241]:
# analysis: hot dog texts
analysis = count_df[(count_df['hot'] + count_df['dogs'])==2]
print(analysis.shape)
# reference: non hot dog texts
reference = count_df[(count_df['hot'] + count_df['dogs'])!=2]
print(reference.shape)
log_likelihood(analysis, reference, 'good')

(4, 23)
(2, 23)
counts analysis: 1
counts reference: 1
G2:  0.21010555200628656


In [243]:
# this is pretty non-significant
# G2=3.84 means 5% of getting this difference by chance, this is closer to 30%
# if we added a few to analysis
analysis['good'] = analysis['good']+3
log_likelihood(analysis, reference, 'good')

counts analysis: 13
counts reference: 1
G2:  3.317957773087898


/opt/anaconda3/envs/mainpy3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## Intro to advanced models
In this section, we'll be setting up some of the requirements for the more advanced techniques we will cover later in the course.  Particularly, we'll be working with:

- [huggingface's transformers library](https://github.com/huggingface/transformers)
- [spaCy-transformers (based on the above)](https://github.com/explosion/spacy-transformers)

These require some additional downloads.  For these examples you'll need:

[BERT uncased large model](https://github.com/google-research/bert)

SpaCy's medium English model (with word vectors from GloVe)


In [ ]:
# install extra models
# only need to run this once per session
#!python -m spacy download en_trf_bertbaseuncased_lg
#!python -m spacy download en_core_web_md
# you will likely need to restart your kernel to load the models
#import spacy

In [ ]:
# load BERT model
nlp_bert = spacy.load("en_trf_bertbaseuncased_lg")
# load medium English model
nlp = spacy.load("en_core_web_md")

In [ ]:
# the spaCy Doc-Span-Token structure is still in place
text = "This is a sentence."
for model in [nlp_bert, nlp]:
    print(model.meta['name'])
    parsed = model(text)
    print(type(parsed), type(parsed[0]))
    # but the vector representation is different
    print('Vector shape:', parsed.vector.shape)
    # documents under the transformer model have additional attributes
    print(parsed._.trf_last_hidden_state)

In [ ]:
# adapted from spaCy's example
# compare two different senses of the word "Apple" based on similarity
# similarity: higher values = more similar
apple_org = "Apple sold fewer iPhones this quarter."
apple_food = "Apple pie is delicious."
for model in [nlp_bert, nlp]:
    print(model.meta['name'])
    print('Similarity between senses:', model(apple_org)[0].similarity(model(apple_food)[0]))

### Additional example: SciSpaCy
I noticed a lot of people in the class are working in pharma and settings where you'd be dealing with scientific text.  When dealing with that, it might make sense to use a model trained on that sort of data.  Enter [scispaCy](https://allenai.github.io/scispacy/)

In [3]:
# you'll need to run these on collab
#!pip install scispacy
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz
# after this, you may need to restart the runtime
import spacy

In [7]:
nlp_sci = spacy.load("en_core_sci_sm")
nlp_web = spacy.load("en_core_web_sm")
text = """
Myeloid derived suppressor cells (MDSC) are immature 
myeloid cells with immunosuppressive activity. 
They accumulate in tumor-bearing mice and humans 
with different types of cancer, including hepatocellular 
carcinoma (HCC).
"""
doc = nlp_sci(text)
print(doc.ents)
doc = nlp_web(text)
print(doc.ents)

(suppressor cells, MDSC, immature, immunosuppressive activity, accumulate, tumor-bearing mice, humans, cancer, hepatocellular 
carcinoma, HCC)
(Myeloid, HCC)
